In [23]:
import json
import datasets
import random
import soundfile as sf

from IPython.display import Audio, display

# StressTest

In [2]:
ds = datasets.load_dataset("slprl/StressTest")

In [3]:
audio = ds['test'][0]['audio'].get_all_samples()

In [9]:
ds['test'][0]['audio'].get_all_samples()

AudioSamples:
  data (shape): torch.Size([1, 58112])
  pts_seconds: 0.0
  duration_seconds: 3.632
  sample_rate: 16000

In [11]:
with open('stresstest/stresstest.jsonl', 'w') as f:
    for data in ds['test']:
        f.write(
            json.dumps({
                'transcription': data['transcription'],
                'description': data['description'],
                'intonation': data['intonation'],
                'metadata': data['metadata'],
                'stress_pattern': data['stress_pattern'],
            }) + '\n'
        )

In [12]:
len(ds['test'])

218

In [ ]:
for idx, data in enumerate(ds['test']):
    random_prompt_idx = random.randint(0, len(ds['test']) - 1)
    prompt_audio = ds['test'][random_prompt_idx]['audio'].get_all_samples()
    sf.write('stresstest/prompt/audio_{}.wav'.format(idx), prompt_audio.data.numpy()[0], prompt_audio.sample_rate)
    ground_truth_audio = data['audio'].get_all_samples()
    sf.write('stresstest/ground_truth/audio_{}.wav'.format(idx), ground_truth_audio.data.numpy()[0], ground_truth_audio.sample_rate)
    break

16000

# ParaSpeechCaps Holdout

In [30]:
holdout = datasets.load_dataset("ajd12342/paraspeechcaps", split="holdout")

README.md: 0.00B [00:00, ?B/s]

data/train_scaled-00000-of-00002.parquet:   0%|          | 0.00/161M [00:00<?, ?B/s]

data/train_scaled-00001-of-00002.parquet:   0%|          | 0.00/161M [00:00<?, ?B/s]

data/train_base-00000-of-00001.parquet:   0%|          | 0.00/37.9M [00:00<?, ?B/s]

data/dev-00000-of-00001.parquet:   0%|          | 0.00/3.94M [00:00<?, ?B/s]

data/holdout-00000-of-00001.parquet:   0%|          | 0.00/4.71M [00:00<?, ?B/s]

Generating train_scaled split:   0%|          | 0/924651 [00:00<?, ? examples/s]

Generating train_base split:   0%|          | 0/116516 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/11967 [00:00<?, ? examples/s]

Generating holdout split:   0%|          | 0/14756 [00:00<?, ? examples/s]

In [36]:
holdout[0]

{'source': 'voxceleb',
 'relative_audio_path': 'voxceleb2/dev/aac/id05998/WNpiaa_BtEc/00264_voicefixer.wav',
 'text_description': [' A Scottish male speaks with clear enunciation and a low-pitched, deep voice. His speech is delivered at a measured speed, and the environment in which the recording was made allows for balanced clarity.'],
 'transcription': " Two separate things. Well, three, actually. You've talked about disability and you've talked about welfare.",
 'intrinsic_tags': ['deep', 'enunciated', 'scottish'],
 'situational_tags': None,
 'basic_tags': ['environment balanced in clarity',
  'low-pitched',
  'male',
  'measured speed'],
 'all_tags': ['deep',
  'enunciated',
  'environment balanced in clarity',
  'low-pitched',
  'male',
  'measured speed',
  'scottish'],
 'speakerid': 'id05998',
 'name': 'Michael Gove',
 'duration': 5.6,
 'gender': 'male',
 'accent': 'scottish',
 'pitch': 'low-pitched',
 'speaking_rate': 'measured speed',
 'noise': 'environment balanced in clarity

In [43]:
emotions = set([
    "enthusiastic",
    "happy",
    "angry",
    "saddened",
    "awed",
    "calm",
    "anxious",
    "disgusted",
    "scared",
    "confused",
    "bored",
    "sleepy",
    "pained",
    "guilt",
    "sarcastic",
    "sympathetic",
    "admiring",
    "desirous"
])

In [44]:
usable_data = []
for data in holdout:
    for tag in data['all_tags']:
        if tag in emotions:
            usable_data.append(
                {
                    'transcription': data['transcription'],
                    'emotion': tag,
                }
            )

In [46]:
with open('paraspeechcaps_holdout.jsonl', 'w') as f:
    for data in usable_data:
        f.write(
            json.dumps(data) + '\n'
        )